In [19]:
import math
from scenedetect import open_video, SceneManager, detect, ContentDetector
from scenedetect.video_splitter import split_video_ffmpeg
import os
import cv2
import json

In [20]:
class VideoObject:
    def __init__(self, threshold, video_path):
        self.threshold = threshold
        self.video_path = video_path
        self.fps = 30
        self.scene_manager = SceneManager()
        self.scene_manager.add_detector(ContentDetector(threshold=self.threshold))
        self.get_video_fps()
        self.video_name = self.get_video_file_name()
        self.scene_list = []

    def get_video_fps(self):
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print("Error: Couldn't open the video.")
            return None
        self.fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        return self.fps
    
    def get_video_file_name(self):
        return self.video_path.split('/')[-1].split('.')[0]

    def split_video(self):
        if not self.scene_list:
            video = open_video(self.video_path)
            self.scene_manager.detect_scenes(video, show_progress=True)
            self.scene_list = self.scene_manager.get_scene_list()
            scene_times = []
            for i in self.scene_list:
                scene_times.append((i[0].get_seconds(), i[1].get_seconds()))
            with open(f"./jsons/{self.video_name}.json", "w") as f:
                json.dump(scene_times, f)
        else:
            print("Video already split")
        return self.scene_list
    
    def save_frames(self):
        if not self.scene_list:
            print("Error: Video not split yet. Call split_video() first.")
            return
        
        # check whether the video is already split
        if os.path.exists(f"./Frames/{self.video_name}"):
            print("Frames already saved.")
            return
        else:
            for i, (start, end) in enumerate(self.scene_list):
                duration = (end.get_seconds() - start.get_seconds())
                num_frames = math.ceil(duration * self.fps)
                frame_time = start.get_seconds() + (duration / 2)
                cap = cv2.VideoCapture(self.video_path)
                cap.set(cv2.CAP_PROP_POS_FRAMES, int(frame_time * self.fps))
                ret, frame = cap.read()
                if ret:
                    # CREATE A FOLDER TO SAVE THE FRAMES
                    if not os.path.exists(f"./Frames/{self.video_name}"):
                        os.makedirs(f"./Frames/{self.video_name}")
                    frame_filename = f"./Frames/{self.video_name}/frame_{i+1}.jpg"
                    cv2.imwrite(frame_filename, frame)
                    print(f"Saved {frame_filename}")
                cap.release()
            print("Finished saving frames.")
            return

In [23]:
# batch processing
def batch_process_videos(videos_list):
    for video in videos_list:
        video_object = VideoObject(threshold=28, video_path=video)
        video_object.split_video()
        video_object.save_frames()
        del video_object
    print("Finished processing all videos.")

In [24]:
# get all the videos in the folder
videos_list = []
for root, dirs, files in os.walk("./Videos"):
    for file in files:
        if file.endswith(".mp4"):
            videos_list.append(os.path.join(root, file))
batch_process_videos(videos_list)

  Detected: 5 | Progress: 100%|██████████| 480/480 [00:00<00:00, 1161.69frames/s]


Frames already saved.


  Detected: 23 | Progress: 100%|██████████| 3232/3232 [00:02<00:00, 1216.99frames/s]


Frames already saved.


[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU type 6
[libdav1d @ 0x10608c7c0] Unknown Metadata OBU 

Frames already saved.
Finished processing all videos.
